## **Memorie di un LLM**

La "memoria" è quella parte di un'applicazione che permette di attributire uno "stato" ad una conversazione con un llm.

Il processo di gestione della memoria si inserisce in modo piuttosto immediato all'interno della pipeline di una chat.

1. Ad ogni nuovo inserimento di testo, le funzionalità di "retrieving" aggiungono lo storico al prompt da passare all'llm.
2. L'llm elabora la risposta.
3. Le funzionalità di storage conservano il nuovo scambio "AI-User".

![image](./memory.png)

L'elemento che varia tra le applicazioni è il livello di complessità con il quale si vuole gestire la memoria. A grandi linee, gli approcci individuabili sono tre:

1. **memoria lineare:** si conservano e rcuperano un numero massimo di n-scambi
2. **memoria non lineare:** l'implementazione dipende dalla dimensione dei dati conservati o  dall'importanza da assegnare a determinate parole o frasi
Nel primo caso avremo una struttura dati che conserva e restituisce una sintesi delle conversazioni. Mentre, nel secondo caso avremo una struttra che conserva l'intera conversazione ma ne restituisce un campionamento, guidato dal testo corrente inserito da un utente
3. **memoria persistente:** la memoria viene gestita attraverso i meccanismi di conservazione e recupero di un dataabase vettoriale.



## **Memoria Lineare**

![image](./linear_mem.jpeg)

In [2]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.llms import ChatMessage, MessageRole


# Inizializza il buffer della memoria con una dimensione massima di 3 messaggi
chat_memory = ChatMemoryBuffer(buffer_size=3, token_limit=3000)

# Simula un'interazione utente-assistente
chat_memory.put(ChatMessage(role=MessageRole.USER,      content=("Ciao! Qual è il meteo oggi?")))
chat_memory.put(ChatMessage(role=MessageRole.ASSISTANT, content=("Oggi è soleggiato con una temperatura di 25°C.")))
chat_memory.put(ChatMessage(role=MessageRole.USER,      content=("Perfetto, e domani?")))
chat_memory.put(ChatMessage(role=MessageRole.ASSISTANT, content=("Domani è prevista pioggia con una temperatura di 18°C.")))


# Recupera tutti i messaggi nel buffer
chat_history = chat_memory.get()

print("Cronologia della conversazione attuale:")
for message in chat_history:
    print(f"{message.role}: {message.content}")

# Aggiungi un nuovo messaggio che fa scattare la rimozione del messaggio più vecchio
chat_memory.put(ChatMessage(role=MessageRole.USER,      content=(("user", "Grazie, allora prendo l'ombrello."))))

# Mostra la nuova cronologia con il messaggio più vecchio eliminato
new_chat_history = chat_memory.get()
print("\nNuova cronologia della conversazione:")
for message in new_chat_history:
    print(f"{message.role}: {message.content}")


Cronologia della conversazione attuale:
MessageRole.USER: Ciao! Qual è il meteo oggi?
MessageRole.ASSISTANT: Oggi è soleggiato con una temperatura di 25°C.
MessageRole.USER: Perfetto, e domani?
MessageRole.ASSISTANT: Domani è prevista pioggia con una temperatura di 18°C.

Nuova cronologia della conversazione:
MessageRole.USER: Ciao! Qual è il meteo oggi?
MessageRole.ASSISTANT: Oggi è soleggiato con una temperatura di 25°C.
MessageRole.USER: Perfetto, e domani?
MessageRole.ASSISTANT: Domani è prevista pioggia con una temperatura di 18°C.
MessageRole.USER: ('user', "Grazie, allora prendo l'ombrello.")


## **Memoria non lineare**

![image](./non_linear_mem.jpeg)

In [ ]:
from llama_index.core.memory import ChatSummaryMemoryBuffer
from llama_index.llms.ollama import Ollama
import tiktoken


llm =  Ollama(model="llama3.2:1b", 
                   temperature=0.5)


memory_sum = ChatSummaryMemoryBuffer.from_defaults(
    llm=llm,
    chat_history=chat_memory.get(),
    token_limit=2)

history = memory_sum.get()
for message in history:
    print(message)


system: <|start_header_id|>assistant<|end_header_id|>

The conversation starts with the user asking for the weather forecast for today and tomorrow, which includes checking if it will be sunny or rainy. The assistant responds by providing the current temperature (25°C) and the forecasted conditions for both days: sunny with 18°C on the next day.


## **Memoria Peristente**

![image](./pers_mem.jpeg)